In [1]:
# Importing the libraries
import numpy as np
import re
import pickle 
import nltk
import pandas as pd
from nltk.corpus import stopwords
from sklearn.datasets import load_files
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Jeremy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
raw_data = pd.read_csv('Hotel_Reviews.csv')

In [3]:
raw_data.head()

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [44]:
df = raw_data.copy()

In [45]:
df['Review'] = df.Negative_Review + df.Positive_Review
df['Review'][2]

' Rooms are nice but for elderly a bit difficult as most rooms are two story with narrow steps So ask for single level Inside the rooms are very very basic just tea coffee and boiler and no bar empty fridge  Location was good and staff were ok It is cute hotel the breakfast range is nice Will go back '

In [46]:
# create the target
df['Target'] = df["Reviewer_Score"].apply(lambda x: 0 if x < 5 else 1)

df = df[['Review', 'Target']]

In [47]:
df.head()

,Review,Target
0,I am so angry that i made this post available...,0
1,No Negative No real complaints the hotel was g...,1
2,Rooms are nice but for elderly a bit difficul...,1
3,My room was dirty and I was afraid to walk ba...,0
4,You When I booked with your company on line y...,1


In [48]:
X, y = df.Review, df.Target

In [49]:
df.shape

(515738, 2)

In [50]:
X.shape

(515738,)

In [51]:
y.shape

(515738,)

In [63]:
#remove excess 0s
one_counter = 0
counter = 0
indices_to_remove =[]


for index, row in df.iterrows():
    if row['Target'] == 1:
        one_counter+=1
        if one_counter >= (df.shape[0] - df.Target.sum()):
            indices_to_remove.append(index)
print('loop done')       
df_balanced = df.drop(indices_to_remove)
df_balanced.reset_index(inplace=True, drop=True)

#check if targets are balance (approx. 50%)

print(df_balanced['Target'].sum())
print(df_balanced['Target'].shape[0])
print(df_balanced['Target'].sum()/df_balanced['Target'].shape[0])

loop done
22280
44561
0.49998877942595543


In [64]:
X = df_balanced['Review']
y = df_balanced['Target']

In [69]:

# Creating the corpus
corpus = []
for i in range(0, len(X)):
    review = re.sub(r'\W', ' ', str(X[i]))
    review = review.lower()
    review = re.sub(r'^br$', ' ', review)
    review = re.sub(r'\s+br\s+',' ',review)
    review = re.sub(r'\s+[a-z]\s+', ' ',review)
    review = re.sub(r'^b\s+', '', review)
    review = re.sub(r'\s+', ' ', review)
    corpus.append(review)

In [70]:
# Creating the Tf-Idf model directly
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features = 2000, min_df = 3, max_df = 0.6, stop_words = stopwords.words('english'))
X = vectorizer.fit_transform(corpus).toarray()

In [71]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
text_train, text_test, sent_train, sent_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [72]:
# Training the classifier
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()
classifier.fit(text_train,sent_train)

LogisticRegression()

In [73]:
# Testing model performance
sent_pred = classifier.predict(text_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(sent_test, sent_pred)

In [74]:
classifier.score(text_test, sent_test)

0.8619993268259846

In [81]:
sample = [" Friendly staff OostPark a few yards away Good continental breakfast selection Quiet and relaxing area "]
sample = vectorizer.transform(sample).toarray()
sentiment = classifier.predict(sample)
sentiment

array([1], dtype=int64)

In [82]:
corpus

[' am so angry that made this post available via all possible sites use when planing my trips so no one will make the mistake of booking this place made my booking via booking com we stayed for 6 nights in this hotel from 11 to 17 july upon arrival we were placed in small room on the 2nd floor of the hotel it turned out that this was not the room we booked had specially reserved the 2 level duplex room so that we would have big windows and high ceilings the room itself was ok if you don mind the broken window that can not be closed hello rain and mini fridge that contained some sort of bio weapon at least guessed so by the smell of it intimately asked to change the room and after explaining 2 times that booked duplex btw it costs the same as simple double but got way more volume due to the high ceiling was offered room but only the next day so had to check out the next day before 11 clock in order to get the room waned to not the best way to begin your holiday so we had to wait till 13